<a href="https://colab.research.google.com/github/Vardhinedi5869ms/PoC/blob/main/AI_Launch_Feasibility_Predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 MB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 87.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.2 MB/s eta 0:00:00


In [3]:
!pip install playsound

  Preparing metadata (setup.py) ... done
  Created wheel for playsound: filename=playsound-1.3.0-py3-none-any.whl size=7020 sha256=34cf7a402cd584fead310f4a54cc95a6c39195be845a587d18800eca2e84da18
  Stored in directory: /root/.cache/pip/wheels/50/98/42/62753a9e1fb97579a0ce2f84f7db4c21c09d03bb2091e6cef4
Successfully built playsound


In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import gradio as gr
import plotly.graph_objects as go


# Set random seed for reproducibility
np.random.seed(42)

# Generate synthetic data with 15 real physics parameters (5000 samples)
num_samples = 5000
time_steps = 1
data = {
    "Temperature_C": np.random.uniform(15, 35, num_samples),
    "Wind_Speed_kmph": np.random.uniform(0, 30, num_samples),
    "Atmospheric_Pressure_hPa": np.random.uniform(990, 1020, num_samples),
    "Humidity_percent": np.random.uniform(20, 90, num_samples),
    "Engine_Thrust_kN": np.random.uniform(500, 2000, num_samples),
    "Fuel_Pump_Pressure_bar": np.random.uniform(50, 200, num_samples),
    "Altitude_m": np.random.uniform(0, 100, num_samples),
    "Solar_Radiation_Wm2": np.random.uniform(0, 1000, num_samples),
    "Precipitation_mmh": np.random.uniform(0, 5, num_samples),
    "Visibility_km": np.random.uniform(5, 20, num_samples),
    "Air_Density_kgm3": np.random.uniform(1.1, 1.225, num_samples),
    "Launch_Angle_deg": np.random.uniform(0, 15, num_samples),
    "Propellant_Temp_C": np.random.uniform(-183, 25, num_samples),
    "Vibration_Hz": np.random.uniform(5, 50, num_samples),
    "Atmospheric_Ozone_ppm": np.random.uniform(0.01, 0.1, num_samples),
}
df = pd.DataFrame(data)

# Physics-based feasibility score
df["Feasibility_Score"] = np.clip(
    0.2 * (df["Temperature_C"] / 35 - df["Wind_Speed_kmph"] / 30) +
    0.2 * (df["Engine_Thrust_kN"] - 500) / 1500 +
    0.15 * (df["Visibility_km"] - 5) / 15 -
    0.15 * (5 - df["Precipitation_mmh"]) / 5 +
    0.1 * (df["Air_Density_kgm3"] - 1.1) / 0.125 +
    0.1 * (15 - df["Launch_Angle_deg"]) / 15 +
    0.1 * ((df["Propellant_Temp_C"] + 183) / 208 - 0.5) -
    0.05 * (50 - df["Vibration_Hz"]) / 45,
    0, 1
)

# Prepare data
X = df.drop(columns=["Feasibility_Score"])
y = df["Feasibility_Score"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define and train model
model = Sequential([
    Dense(256, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    Dropout(0.2),
    Dense(128, activation='relu'),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
])
model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
history = model.fit(
    X_train_scaled, y_train,
    validation_split=0.2,
    epochs=50,
    batch_size=32,
    callbacks=[EarlyStopping(patience=10), ReduceLROnPlateau(factor=0.2, patience=5)],
    verbose=0
)

# Evaluate
y_pred = model.predict(X_test_scaled, verbose=0)
print(f"MAE: {mean_absolute_error(y_test, y_pred):.4f}, R²: {r2_score(y_test, y_pred):.4f}")

# Thresholds and realistic ranges
thresholds = {
    "Temperature_C": {"min": 15, "max": 35, "realistic_max": 35},
    "Wind_Speed_kmph": {"max": 20, "realistic_max": 30},
    "Atmospheric_Pressure_hPa": {"min": 995, "max": 1015, "realistic_max": 1020},
    "Humidity_percent": {"max": 85, "realistic_max": 90},
    "Engine_Thrust_kN": {"min": 600, "realistic_max": 2000},
    "Fuel_Pump_Pressure_bar": {"min": 70, "realistic_max": 200},
    "Altitude_m": {"max": 100, "realistic_max": 100},
    "Solar_Radiation_Wm2": {"max": 800, "realistic_max": 1000},
    "Precipitation_mmh": {"max": 2, "realistic_max": 5},
    "Visibility_km": {"min": 10, "realistic_max": 20},
    "Air_Density_kgm3": {"min": 1.15, "max": 1.225, "realistic_max": 1.225},
    "Launch_Angle_deg": {"max": 10, "realistic_max": 15},
    "Propellant_Temp_C": {"min": -183, "max": 20, "realistic_max": 25},
    "Vibration_Hz": {"max": 30, "realistic_max": 50},
    "Atmospheric_Ozone_ppm": {"max": 0.08, "realistic_max": 0.1},
}

# Sidebar content
threshold_text = "\n".join(
    f"{k}: {thresholds[k].get('min', '-')} to {thresholds[k].get('max', '-')}"
    for k in thresholds.keys()
)

# Prediction and analysis function with all parameter trends
def analyze_mission_data(temp, wind, pressure, humidity, thrust, pump_pressure, altitude, solar, precip, visibility, air_density, launch_angle, prop_temp, vibration, ozone):
    inputs = {
        "Temperature_C": min(max(temp, 15), thresholds["Temperature_C"]["realistic_max"]),
        "Wind_Speed_kmph": min(max(wind, 0), thresholds["Wind_Speed_kmph"]["realistic_max"]),
        "Atmospheric_Pressure_hPa": min(max(pressure, 990), thresholds["Atmospheric_Pressure_hPa"]["realistic_max"]),
        "Humidity_percent": min(max(humidity, 20), thresholds["Humidity_percent"]["realistic_max"]),
        "Engine_Thrust_kN": min(max(thrust, 500), thresholds["Engine_Thrust_kN"]["realistic_max"]),
        "Fuel_Pump_Pressure_bar": min(max(pump_pressure, 50), thresholds["Fuel_Pump_Pressure_bar"]["realistic_max"]),
        "Altitude_m": min(max(altitude, 0), thresholds["Altitude_m"]["realistic_max"]),
        "Solar_Radiation_Wm2": min(max(solar, 0), thresholds["Solar_Radiation_Wm2"]["realistic_max"]),
        "Precipitation_mmh": min(max(precip, 0), thresholds["Precipitation_mmh"]["realistic_max"]),
        "Visibility_km": min(max(visibility, 5), thresholds["Visibility_km"]["realistic_max"]),
        "Air_Density_kgm3": min(max(air_density, 1.1), thresholds["Air_Density_kgm3"]["realistic_max"]),
        "Launch_Angle_deg": min(max(launch_angle, 0), thresholds["Launch_Angle_deg"]["realistic_max"]),
        "Propellant_Temp_C": min(max(prop_temp, -183), thresholds["Propellant_Temp_C"]["realistic_max"]),
        "Vibration_Hz": min(max(vibration, 5), thresholds["Vibration_Hz"]["realistic_max"]),
        "Atmospheric_Ozone_ppm": min(max(ozone, 0.01), thresholds["Atmospheric_Ozone_ppm"]["realistic_max"]),
    }

    input_data = np.array([[inputs[k] for k in inputs.keys()]])
    input_scaled = scaler.transform(input_data)
    score = model.predict(input_scaled, verbose=0)[0][0]

    violations = {
        k: inputs[k] < thresholds[k].get("min", -float('inf')) or inputs[k] > thresholds[k].get("max", float('inf'))
        for k in inputs.keys()
    }
    violation_text = [
        f"{k}: {inputs[k]:.1f} (limit: {thresholds[k].get('min', '-')} to {thresholds[k].get('max', '-')})"
        for k in inputs.keys() if violations[k]
    ]
    warnings = [
        f"{k}: {v:.1f} exceeds realistic range (max {thresholds[k]['realistic_max']})"
        for k, v in {"Temperature_C": temp, "Wind_Speed_kmph": wind, "Atmospheric_Pressure_hPa": pressure,
                     "Humidity_percent": humidity, "Engine_Thrust_kN": thrust, "Fuel_Pump_Pressure_bar": pump_pressure,
                     "Altitude_m": altitude, "Solar_Radiation_Wm2": solar, "Precipitation_mmh": precip, "Visibility_km": visibility,
                     "Air_Density_kgm3": air_density, "Launch_Angle_deg": launch_angle, "Propellant_Temp_C": prop_temp,
                     "Vibration_Hz": vibration, "Atmospheric_Ozone_ppm": ozone}.items()
        if v > thresholds[k]["realistic_max"]
    ]

    decision = "Optimal" if score >= 0.7 else "Suboptimal"
    insights = f"Feasibility Score: {score:.4f}\nDecision: {decision}"
    insights += f"\nInputs: Temp={inputs['Temperature_C']:.1f}, Wind={inputs['Wind_Speed_kmph']:.1f}, Pressure={inputs['Atmospheric_Pressure_hPa']:.1f}, Humidity={inputs['Humidity_percent']:.1f}, Thrust={inputs['Engine_Thrust_kN']:.1f}, Pump={inputs['Fuel_Pump_Pressure_bar']:.1f}, Altitude={inputs['Altitude_m']:.1f}, Solar={inputs['Solar_Radiation_Wm2']:.1f}, Precip={inputs['Precipitation_mmh']:.1f}, Visibility={inputs['Visibility_km']:.1f}, AirDensity={inputs['Air_Density_kgm3']:.2f}, LaunchAngle={inputs['Launch_Angle_deg']:.1f}, PropTemp={inputs['Propellant_Temp_C']:.1f}, Vibration={inputs['Vibration_Hz']:.1f}, Ozone={inputs['Atmospheric_Ozone_ppm']:.2f}"
    if warnings:
        insights += f"\nWarnings: {', '.join(warnings)}"
    if violation_text:
        insights += f"\nIssues: {', '.join(violation_text)}"
    else:
        if score < 0.7:
            impacts = {k: abs(v - np.mean(X_train[k])) / np.std(X_train[k]) for k, v in inputs.items()}
            significant = {k: v for k, v in impacts.items() if v > 1.0}
            if significant:
                key_factor = max(significant, key=significant.get)
                insights += f"\nNote: Low score likely due to {key_factor} (value: {inputs[key_factor]:.1f})."
            else:
                insights += "\nNote: Low score due to combined factors; no single parameter stands out."
            if score < 0.3:
                insights += "\n🚨 ALERT: Critical conditions detected! 🚨"
        else:
            insights += "\nConditions ideal due to favorable weather, thrust, and stability."

    # Bar chart for current values
    fig_bar = go.Figure()
    categories = list(inputs.keys())
    values = [inputs[k] for k in categories]
    fig_bar.add_trace(go.Bar(x=categories, y=values, name="Current Values"))
    fig_bar.update_layout(title="Input Parameters Overview", yaxis_title="Value", template="plotly_dark")
    for param, thresh in thresholds.items():
        if "min" in thresh:
            fig_bar.add_hline(y=thresh["min"], line_dash="dash", line_color="red", annotation_text=f"{param} Min")
        if "max" in thresh:
            fig_bar.add_hline(y=thresh["max"], line_dash="dash", line_color="red", annotation_text=f"{param} Max")

    # Time-series plot for all 15 parameters (normalized)
    time_minutes = [0, 1, 2, 3, 4]
    trends = {}
    for param in inputs.keys():
        min_val = thresholds[param].get("min", inputs[param] - 10)  # Default min if not specified
        max_val = thresholds[param]["realistic_max"]
        variation = (max_val - min_val) * 0.05  # 5% variation
        trends[param] = [inputs[param] + np.random.uniform(-variation, variation) for _ in range(5)]
        # Normalize to 0-1
        trends[param] = [(x - min_val) / (max_val - min_val) for x in trends[param]]

    fig_time = go.Figure()
    for param, trend in trends.items():
        fig_time.add_trace(go.Scatter(x=time_minutes, y=trend, mode='lines+markers', name=param))
    fig_time.update_layout(
        title="Parameter Trends (Last 5 Minutes, Normalized 0-1)",
        xaxis_title="Time (minutes)",
        yaxis_title="Normalized Value",
        template="plotly_dark",
        legend=dict(orientation="h", yanchor="bottom", y=-0.5, xanchor="center", x=0.5)
    )

    # Update input box styles
    input_updates = [
        gr.update(elem_classes="non-optimal" if violations["Temperature_C"] else ""),
        gr.update(elem_classes="non-optimal" if violations["Wind_Speed_kmph"] else ""),
        gr.update(elem_classes="non-optimal" if violations["Atmospheric_Pressure_hPa"] else ""),
        gr.update(elem_classes="non-optimal" if violations["Humidity_percent"] else ""),
        gr.update(elem_classes="non-optimal" if violations["Engine_Thrust_kN"] else ""),
        gr.update(elem_classes="non-optimal" if violations["Fuel_Pump_Pressure_bar"] else ""),
        gr.update(elem_classes="non-optimal" if violations["Altitude_m"] else ""),
        gr.update(elem_classes="non-optimal" if violations["Solar_Radiation_Wm2"] else ""),
        gr.update(elem_classes="non-optimal" if violations["Precipitation_mmh"] else ""),
        gr.update(elem_classes="non-optimal" if violations["Visibility_km"] else ""),
        gr.update(elem_classes="non-optimal" if violations["Air_Density_kgm3"] else ""),
        gr.update(elem_classes="non-optimal" if violations["Launch_Angle_deg"] else ""),
        gr.update(elem_classes="non-optimal" if violations["Propellant_Temp_C"] else ""),
        gr.update(elem_classes="non-optimal" if violations["Vibration_Hz"] else ""),
        gr.update(elem_classes="non-optimal" if violations["Atmospheric_Ozone_ppm"] else ""),
    ]

    return [insights, fig_bar, fig_time, threshold_text] + input_updates

# Gradio interface with sidebar, bar chart, and all parameter trends
with gr.Blocks(title="Space Mission Data Analytics Dashboard", css=".non-optimal {border: 2px solid red !important;}") as interface:
    gr.Markdown("### Space Mission Data Analytics Dashboard")
    gr.Markdown("Enter real physics-based launch data to assess mission feasibility. Non-optimal inputs turn red. ALERT triggers if score < 0.3.")

    with gr.Row():
        with gr.Column(scale=1):
            gr.Markdown("### Thresholds")
            threshold_display = gr.Textbox(value=threshold_text, label="Acceptable Ranges", interactive=False)

        with gr.Column(scale=3):
            temp_input = gr.Number(label="Temperature (°C)", value=25)
            wind_input = gr.Number(label="Wind Speed (kmph)", value=10)
            pressure_input = gr.Number(label="Atmospheric Pressure (hPa)", value=1010)
            humidity_input = gr.Number(label="Humidity (%)", value=50)
            thrust_input = gr.Number(label="Engine Thrust (kN)", value=1500)
            pump_input = gr.Number(label="Fuel Pump Pressure (bar)", value=100)
            altitude_input = gr.Number(label="Altitude (m)", value=50)
            solar_input = gr.Number(label="Solar Radiation (W/m²)", value=500)
            precip_input = gr.Number(label="Precipitation (mm/h)", value=0)
            visibility_input = gr.Number(label="Visibility (km)", value=15)
            air_density_input = gr.Number(label="Air Density (kg/m³)", value=1.2)
            launch_angle_input = gr.Number(label="Launch Angle (deg)", value=5)
            prop_temp_input = gr.Number(label="Propellant Temp (°C)", value=-150)
            vibration_input = gr.Number(label="Vibration (Hz)", value=20)
            ozone_input = gr.Number(label="Atmospheric Ozone (ppm)", value=0.05)
            submit_btn = gr.Button("Analyze")

            insights_output = gr.Textbox(label="Mission Insights")
            bar_plot_output = gr.Plot(label="Parameter Visualization")
            time_plot_output = gr.Plot(label="All Parameter Trends (Normalized)")

    submit_btn.click(
        fn=analyze_mission_data,
        inputs=[temp_input, wind_input, pressure_input, humidity_input, thrust_input, pump_input, altitude_input, solar_input, precip_input, visibility_input, air_density_input, launch_angle_input, prop_temp_input, vibration_input, ozone_input],
        outputs=[insights_output, bar_plot_output, time_plot_output, threshold_display, temp_input, wind_input, pressure_input, humidity_input, thrust_input, pump_input, altitude_input, solar_input, precip_input, visibility_input, air_density_input, launch_angle_input, prop_temp_input, vibration_input, ozone_input]
    )

interface.launch()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


MAE: 0.0057, R²: 0.9955
Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://86ab4a9f73c64cd6ef.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
